In [1]:
import os
import sys
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import json
import importlib.util
from torch.utils.data import Dataset, DataLoader

# ==========================================
# 1. CẤU HÌNH ĐƯỜNG DẪN (ĐÃ CẬP NHẬT TỪ KẾT QUẢ QUÉT)
# ==========================================

# Đường dẫn file chứa code model (Lấy từ kết quả scan của bạn)
MODEL_CODE_FILE = r"D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\SpliceTransformer\SpliceTransformer-main\model\model.py"

# Tên Class Model (Lấy từ kết quả scan)
MODEL_CLASS_NAME = "SpTransformer"

# Đường dẫn file weights (.ckpt)
CKPT_PATH = r"D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\SpliceTransformer\model.ckpt"

# Thư mục chứa 4 file csv prepared
DATA_DIR = r"D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\data_test"

# Thư mục xuất kết quả
RESULT_DIR = r"D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\results"

BATCH_SIZE = 16
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
ratios = ["1_1_1", "2_1_1", "4_1_1", "10_1_1", "100_1_1"]

# ==========================================
# 2. HÀM LOAD MODEL TỪ FILE BẤT KỲ
# ==========================================
def load_model_class(file_path, class_name):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"❌ Không tìm thấy file code tại: {file_path}")
    
    # Load module từ đường dẫn file
    spec = importlib.util.spec_from_file_location("dynamic_model_module", file_path)
    module = importlib.util.module_from_spec(spec)
    sys.modules["dynamic_model_module"] = module
    spec.loader.exec_module(module)
    
    if hasattr(module, class_name):
        print(f"✅ Đã import thành công class '{class_name}' từ {os.path.basename(file_path)}")
        return getattr(module, class_name)
    else:
        raise AttributeError(f"❌ Trong file '{file_path}' không có class '{class_name}'")

# ==========================================
# 3. HÀM LOAD CHECKPOINT (.ckpt / .pth)
# ==========================================
def load_weights(model, ckpt_path):
    print(f"🔄 Đang load weights từ: {ckpt_path}")
    
    # Load checkpoint lên CPU/GPU
    checkpoint = torch.load(ckpt_path, map_location=DEVICE)
    
    # Trích xuất state_dict
    if isinstance(checkpoint, dict):
        if 'state_dict' in checkpoint:
            state_dict = checkpoint['state_dict']
        elif 'model_state_dict' in checkpoint:
            state_dict = checkpoint['model_state_dict']
        else:
            state_dict = checkpoint
    else:
        state_dict = checkpoint

    # Xử lý tên keys (xóa prefix thừa)
    clean_state_dict = {}
    for key, val in state_dict.items():
        new_key = key
        # Xóa các tiền tố phổ biến do PyTorch Lightning hoặc DataParallel sinh ra
        for prefix in ["model.", "net.", "module.", "backbone."]:
            if new_key.startswith(prefix):
                new_key = new_key[len(prefix):]
                break
        clean_state_dict[new_key] = val

    # Load vào model
    try:
        model.load_state_dict(clean_state_dict, strict=False)
        print("✅ Load weights hoàn tất!")
    except Exception as e:
        print(f"⚠️ Cảnh báo khi load weights: {e}")

# ==========================================
# 4. DATASET & METRICS
# ==========================================
# Fallback metrics nếu thiếu file metrics.py
try:
    from metrics import compute_metrics
except ImportError:
    print("⚠️ Không tìm thấy file 'metrics.py'. Sẽ dùng hàm metrics đơn giản.")
    def compute_metrics(labels, preds, probs=None, k=2):
        # Tính accuracy đơn giản để không bị lỗi code
        acc = (labels == preds).mean()
        return {"accuracy": float(acc), "note": "metrics.py not found"}

class InferenceDataset(Dataset):
    def __init__(self, csv_path):
        self.df = pd.read_csv(csv_path)
        self.mapping = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        # Lấy sequence và label
        seq_str = str(self.df.iloc[idx]['sequence']).upper()
        label = int(self.df.iloc[idx]['Splicing_types'])
        
        # One-hot encoding
        seq_enc = np.zeros((len(seq_str), 4), dtype=np.float32)
        for i, char in enumerate(seq_str):
            if char in self.mapping:
                seq_enc[i, self.mapping[char]] = 1.0
            else:
                seq_enc[i] = 0.25
        
        # Transpose (L, 4) -> (4, L) cho Model
        return torch.tensor(seq_enc).transpose(0, 1), torch.tensor(label, dtype=torch.long)

# ==========================================
# 5. MAIN RUN
# ==========================================
def main():
    if not os.path.exists(RESULT_DIR):
        os.makedirs(RESULT_DIR)
        
    try:
        # 1. Load Class Model từ file model/model.py
        ModelClass = load_model_class(MODEL_CODE_FILE, MODEL_CLASS_NAME)
        
        # 2. Khởi tạo Model
        # Lưu ý: Nếu __init__ của model yêu cầu tham số, cần truyền vào đây. 
        # Thường SpTransformer mặc định params chuẩn.
        try:
            model = ModelClass().to(DEVICE)
        except TypeError as e:
            print(f"⚠️ Model yêu cầu tham số khởi tạo: {e}")
            print("👉 Đang thử khởi tạo với tham số mặc định (nếu cần)...")
            # Ví dụ: model = ModelClass(maxlen=...)
            model = ModelClass().to(DEVICE)

        # 3. Load Weights
        if os.path.exists(CKPT_PATH):
            load_weights(model, CKPT_PATH)
        else:
            print(f"❌ Không tìm thấy file weights tại: {CKPT_PATH}")
            return
            
        model.eval()
        
        # 4. Chạy vòng lặp Inference
        for ratio in ratios:
            csv_path = os.path.join(DATA_DIR, f"prepared_inference_{ratio}.csv")
            if not os.path.exists(csv_path):
                print(f"⏩ Bỏ qua {ratio}: File không tồn tại")
                continue
                
            print(f"\n>>> 🏃 Đang chạy: {ratio}")
            dataset = InferenceDataset(csv_path)
            loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False)
            
            all_labels, all_preds, all_probs = [], [], []
            
            with torch.no_grad():
                for inputs, labels in loader:
                    inputs = inputs.to(DEVICE)
                    
                    # Forward pass
                    outputs = model(inputs)
                    
                    # Xử lý output nếu là tuple/list
                    if isinstance(outputs, (tuple, list)):
                        outputs = outputs[0]
                    
                    # Tính xác suất
                    probs = torch.softmax(outputs, dim=1)
                    preds = torch.argmax(probs, dim=1)
                    
                    all_labels.extend(labels.cpu().numpy())
                    all_preds.extend(preds.cpu().numpy())
                    all_probs.extend(probs.cpu().numpy())
            
            # Tính và lưu kết quả
            results = compute_metrics(np.array(all_labels), np.array(all_preds), np.array(all_probs))
            
            out_file = os.path.join(RESULT_DIR, f"results_{ratio}.json")
            with open(out_file, 'w') as f:
                json.dump(results, f, indent=4)
                
            print(f"   ✅ Hoàn tất. Accuracy: {results.get('accuracy', 0):.4f}")
            print(f"   📂 Kết quả lưu tại: {out_file}")

    except Exception as e:
        print(f"\n❌ LỖI TRONG QUÁ TRÌNH CHẠY: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

⚠️ Không tìm thấy file 'metrics.py'. Sẽ dùng hàm metrics đơn giản.
✅ Đã import thành công class 'SpTransformer' từ model.py
❌ Không tìm thấy file weights tại: D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\SpliceTransformer\model.ckpt


In [1]:
import os
import sys
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import json
import importlib.util
from torch.utils.data import Dataset, DataLoader

# ==========================================
# 1. CẤU HÌNH ĐƯỜNG DẪN (ĐÃ GIỮ NGUYÊN CỦA BẠN)
# ==========================================

METRICS_FILE_PATH = r"D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\SpliceTransformer\metrics.py"
MODEL_CODE_FILE = r"D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\SpliceTransformer\SpliceTransformer-main\model\model.py"
MODEL_CLASS_NAME = "SpTransformer"
CKPT_PATH = r"D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\SpliceTransformer\SpTransformer_pytorch.ckpt"
DATA_DIR = r"D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\SpliceTransformer\data"
RESULT_DIR = r"D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\SpliceTransformer\results"

# --- CẤU HÌNH MỚI: ĐỘ DÀI CHUỖI ---
# SpliceTransformer thường yêu cầu 5000 hoặc 10000. 
# Nếu chạy vẫn lỗi kích thước, hãy thử đổi số này thành 10000.
MAX_SEQ_LEN = 10000  

BATCH_SIZE = 16
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
ratios = ["1_1_1", "2_1_1", "4_1_1", "10_1_1", "100_1_1"]

# ==========================================
# 2. HÀM LOAD MODULE
# ==========================================
def load_module_from_path(file_path, module_name="custom_module"):
    """Load một file python như một module từ đường dẫn tuyệt đối"""
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"❌ Không tìm thấy file tại: {file_path}")
    
    spec = importlib.util.spec_from_file_location(module_name, file_path)
    if spec is None:
        raise ImportError(f"❌ Không thể đọc specs từ file: {file_path}")
        
    module = importlib.util.module_from_spec(spec)
    sys.modules[module_name] = module
    spec.loader.exec_module(module)
    return module

# ==========================================
# 3. HÀM LOAD WEIGHTS THÔNG MINH
# ==========================================
def load_weights_smart(model, ckpt_path):
    print(f"🔄 Đang load weights từ: {ckpt_path}")
    checkpoint = torch.load(ckpt_path, map_location=DEVICE)
    
    # 1. Lấy state_dict
    if isinstance(checkpoint, dict):
        if 'state_dict' in checkpoint: state_dict = checkpoint['state_dict']
        elif 'model_state_dict' in checkpoint: state_dict = checkpoint['model_state_dict']
        else: state_dict = checkpoint
    else:
        state_dict = checkpoint

    # 2. Xóa prefix thừa
    clean_state_dict = {}
    for key, val in state_dict.items():
        new_key = key
        for prefix in ["model.", "net.", "module.", "backbone."]:
            if new_key.startswith(prefix):
                new_key = new_key[len(prefix):]
                break
        clean_state_dict[new_key] = val

    # 3. Load vào model
    try:
        model.load_state_dict(clean_state_dict, strict=False)
        print("✅ Load weights thành công!")
    except Exception as e:
        print(f"⚠️ Cảnh báo load weight: {e}")

# ==========================================
# 4. DATASET (ĐÃ SỬA: THÊM PADDING/CROP)
# ==========================================
class InferenceDataset(Dataset):
    def __init__(self, csv_path, max_len=MAX_SEQ_LEN):
        self.df = pd.read_csv(csv_path)
        self.mapping = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
        self.max_len = max_len
        
    def __len__(self): return len(self.df)
    
    def __getitem__(self, idx):
        # Lấy chuỗi và chuẩn hóa
        seq = str(self.df.iloc[idx]['sequence']).upper().strip()
        label = int(self.df.iloc[idx]['Splicing_types'])
        
        curr_len = len(seq)
        
        # --- XỬ LÝ ĐỘ DÀI (PADDING HOẶC CROP) ---
        if curr_len < self.max_len:
            # Nếu ngắn hơn -> Thêm N vào 2 đầu (để tâm ở giữa)
            pad_total = self.max_len - curr_len
            pad_left = pad_total // 2
            pad_right = pad_total - pad_left
            seq = "N" * pad_left + seq + "N" * pad_right
        elif curr_len > self.max_len:
            # Nếu dài hơn -> Cắt lấy đoạn giữa
            start = (curr_len - self.max_len) // 2
            seq = seq[start : start + self.max_len]
            
        # --- ONE-HOT ENCODING ---
        seq_enc = np.zeros((len(seq), 4), dtype=np.float32)
        for i, base in enumerate(seq):
            if base in self.mapping: 
                seq_enc[i, self.mapping[base]] = 1.0
            else: 
                seq_enc[i] = 0.25 # Giá trị cho N
        
        # Transpose (L, 4) -> (4, L)
        return torch.tensor(seq_enc).transpose(0, 1), torch.tensor(label, dtype=torch.long)

# ==========================================
# 5. CHƯƠNG TRÌNH CHÍNH
# ==========================================
def main():
    if not os.path.exists(RESULT_DIR): os.makedirs(RESULT_DIR)

    print("🚀 BẮT ĐẦU CHẠY INFERENCE...")

    # A. LOAD METRICS
    try:
        metrics_mod = load_module_from_path(METRICS_FILE_PATH, "metrics_mod")
        compute_metrics = metrics_mod.compute_metrics
        print(f"✅ Đã load metrics từ: {METRICS_FILE_PATH}")
    except Exception as e:
        print(f"⚠️ Lỗi load metrics.py: {e}")
        print("👉 Chuyển sang chế độ tính Accuracy đơn giản (không cần sklearn)")
        def compute_metrics(labels, preds, probs=None, k=2):
            return {"accuracy": float((labels == preds).mean()), "note": "fallback mode"}

    # B. LOAD MODEL & WEIGHTS
    try:
        model_mod = load_module_from_path(MODEL_CODE_FILE, "model_mod")
        
        if not hasattr(model_mod, MODEL_CLASS_NAME):
            print(f"❌ Không tìm thấy class '{MODEL_CLASS_NAME}'")
            return
            
        ModelClass = getattr(model_mod, MODEL_CLASS_NAME)
        
        # Khởi tạo model (Thử với tham số độ dài nếu cần)
        try:
            model = ModelClass().to(DEVICE)
        except:
            print(f"ℹ️ Init mặc định lỗi, thử init với L={MAX_SEQ_LEN}...")
            model = ModelClass(L=MAX_SEQ_LEN).to(DEVICE)

        load_weights_smart(model, CKPT_PATH)
        model.eval()

    except Exception as e:
        print(f"❌ Lỗi khởi tạo Model: {e}")
        return

    # C. CHẠY VÒNG LẶP
    for ratio in ratios:
        csv_path = os.path.join(DATA_DIR, f"prepared_inference_{ratio}.csv")
        if not os.path.exists(csv_path):
            print(f"⏩ Bỏ qua {ratio}: File không tồn tại")
            continue
            
        print(f"\n>>> 🏃 Đang xử lý: {ratio} (Len: {MAX_SEQ_LEN})")
        
        # Truyền max_len vào Dataset
        dataset = InferenceDataset(csv_path, max_len=MAX_SEQ_LEN)
        loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False)
        
        all_labels, all_preds, all_probs = [], [], []

        with torch.no_grad():
            for inputs, labels in loader:
                inputs = inputs.to(DEVICE)
                
                outputs = model(inputs)
                if isinstance(outputs, (tuple, list)): outputs = outputs[0]
                
                probs = torch.softmax(outputs, dim=1)
                preds = torch.argmax(probs, dim=1)

                all_labels.extend(labels.cpu().numpy())
                all_preds.extend(preds.cpu().numpy())
                all_probs.extend(probs.cpu().numpy())

        # Tính Metrics
        try:
            results = compute_metrics(np.array(all_labels), np.array(all_preds), np.array(all_probs))
            
            out_file = os.path.join(RESULT_DIR, f"results_{ratio}.json")
            with open(out_file, 'w') as f:
                json.dump(results, f, indent=4)
            print(f"   ✅ Xong. Acc: {results.get('accuracy', 0):.4f} | Saved: {out_file}")
            
        except Exception as e:
            print(f"❌ Lỗi tính metrics cho {ratio}: {e}")

if __name__ == "__main__":
    main()

🚀 BẮT ĐẦU CHẠY INFERENCE...
✅ Đã load metrics từ: D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\SpliceTransformer\metrics.py
🔄 Đang load weights từ: D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\SpliceTransformer\SpTransformer_pytorch.ckpt
⚠️ Cảnh báo load weight: Error(s) in loading state_dict for SpTransformer:
	size mismatch for conv1.0.weight: copying a param with shape torch.Size([128, 4, 1]) from checkpoint, the shape in current model is torch.Size([32, 4, 1]).
	size mismatch for conv1.0.bias: copying a param with shape torch.Size([128]) from checkpoint, the shape in current model is torch.Size([32]).
	size mismatch for conv1.1.weight: copying a param with shape torch.Size([128, 128, 1]) from checkpoint, the shape in current model is torch.Size([32, 32, 1]).
	size mismatch for conv1.1.bias: copying a param with shape torch.Size([128]) from checkpoint, the shape in current model is torch.Size([32]).
	size mismatch for conv2.weight: copying a param with shape torc

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_24052\288485761.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_location=DEVICE)


KeyboardInterrupt: 

In [3]:
import os
import ast

# Đường dẫn thư mục gốc của repo bạn đã tải
SEARCH_DIR = r"D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\SpliceTransformer\SpliceTransformer-main"

print(f"🔍 Đang quét tìm Model trong: {SEARCH_DIR}...\n")

found_any = False

for root, dirs, files in os.walk(SEARCH_DIR):
    for file in files:
        if file.endswith(".py"):
            full_path = os.path.join(root, file)
            try:
                with open(full_path, "r", encoding="utf-8", errors="ignore") as f:
                    file_content = f.read()
                    
                    # Dùng AST để phân tích code mà không cần import (tránh lỗi thiếu thư viện)
                    try:
                        tree = ast.parse(file_content)
                    except SyntaxError:
                        continue # Bỏ qua file lỗi cú pháp

                    for node in ast.walk(tree):
                        if isinstance(node, ast.ClassDef):
                            is_model = False
                            # Kiểm tra xem class có kế thừa nn.Module hay LightningModule không
                            for base in node.bases:
                                # Case 1: class Model(nn.Module) -> base là Attribute
                                if isinstance(base, ast.Attribute) and base.attr in ['Module', 'LightningModule']:
                                    is_model = True
                                # Case 2: class Model(Module) -> base là Name
                                elif isinstance(base, ast.Name) and base.id in ['Module', 'LightningModule']:
                                    is_model = True
                            
                            if is_model:
                                print(f"✅ TÌM THẤY ỨNG VIÊN!")
                                print(f"   📂 File: {full_path}")
                                print(f"   🧩 Class Name: {node.name}")
                                print("-" * 50)
                                found_any = True
            except Exception as e:
                pass

if not found_any:
    print("❌ Vẫn không tìm thấy class nào kế thừa Module. Bạn có chắc đã tải đủ code không?")

🔍 Đang quét tìm Model trong: D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\SpliceTransformer\SpliceTransformer-main...

✅ TÌM THẤY ỨNG VIÊN!
   📂 File: D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\SpliceTransformer\SpliceTransformer-main\model\model.py
   🧩 Class Name: ResBlock
--------------------------------------------------
✅ TÌM THẤY ỨNG VIÊN!
   📂 File: D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\SpliceTransformer\SpliceTransformer-main\model\model.py
   🧩 Class Name: SpEncoder
--------------------------------------------------
✅ TÌM THẤY ỨNG VIÊN!
   📂 File: D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\SpliceTransformer\SpliceTransformer-main\model\model.py
   🧩 Class Name: SpEncoder_4tis
--------------------------------------------------
✅ TÌM THẤY ỨNG VIÊN!
   📂 File: D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\SpliceTransformer\SpliceTransformer-main\model\model.py
   🧩 Class Name: AttnBlock
----------------------------------